# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
bodies = pd.read_csv('/content/drive/My Drive/Fake News Challenge/train_bodies.csv')

In [9]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [0]:
stances=pd.read_csv('/content/drive/My Drive/Fake News Challenge/train_stances.csv')

In [11]:
stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
dataset=pd.merge(bodies,stances,on='Body ID')

In [13]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [14]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [17]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t=Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(dataset['articleBody'])

In [21]:
t.document_count

49972

In [90]:
len(t.word_index)

27873

In [23]:
t.word_counts

OrderedDict([('a', 471875),
             ('small', 4923),
             ('meteorite', 4597),
             ('crashed', 207),
             ('into', 24822),
             ('wooded', 554),
             ('area', 7121),
             ('in', 394397),
             ("nicaragua's", 403),
             ('capital', 6389),
             ('of', 432925),
             ('managua', 1483),
             ('overnight', 749),
             ('the', 1122054),
             ('government', 23143),
             ('said', 133986),
             ('sunday', 5416),
             ('residents', 3383),
             ('reported', 21615),
             ('hearing', 2460),
             ('mysterious', 725),
             ('boom', 1398),
             ('that', 232853),
             ('left', 11624),
             ('16', 2092),
             ('foot', 2312),
             ('deep', 1159),
             ('crater', 3166),
             ('near', 12473),
             ("city's", 726),
             ('airport', 5122),
             ('associated', 4582),
  

In [22]:
t.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'in': 5,
 'and': 6,
 'that': 7,
 'is': 8,
 'was': 9,
 'on': 10,
 'for': 11,
 'said': 12,
 'he': 13,
 'with': 14,
 'it': 15,
 'his': 16,
 'have': 17,
 'as': 18,
 'by': 19,
 'has': 20,
 'from': 21,
 'at': 22,
 'be': 23,
 'an': 24,
 'not': 25,
 'are': 26,
 'been': 27,
 '”': 28,
 'but': 29,
 'this': 30,
 'had': 31,
 'who': 32,
 'they': 33,
 'after': 34,
 'i': 35,
 'were': 36,
 'we': 37,
 'will': 38,
 'about': 39,
 'one': 40,
 'or': 41,
 'which': 42,
 'she': 43,
 'video': 44,
 'apple': 45,
 'up': 46,
 'would': 47,
 'her': 48,
 'state': 49,
 'their': 50,
 'also': 51,
 'more': 52,
 'when': 53,
 'told': 54,
 'out': 55,
 'isis': 56,
 'all': 57,
 'no': 58,
 'new': 59,
 'people': 60,
 'there': 61,
 'you': 62,
 'its': 63,
 'if': 64,
 'him': 65,
 'news': 66,
 'what': 67,
 'could': 68,
 'man': 69,
 'year': 70,
 'islamic': 71,
 'time': 72,
 'some': 73,
 'al': 74,
 'according': 75,
 'watch': 76,
 'over': 77,
 'group': 78,
 'into': 79,
 'so': 80,
 'first': 81,
 

In [0]:
t.fit_on_texts(dataset['Headline'])

In [26]:
t.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'in': 5,
 'and': 6,
 'that': 7,
 'is': 8,
 'was': 9,
 'on': 10,
 'for': 11,
 'said': 12,
 'he': 13,
 'with': 14,
 'it': 15,
 'his': 16,
 'have': 17,
 'as': 18,
 'by': 19,
 'has': 20,
 'at': 21,
 'from': 22,
 'be': 23,
 'an': 24,
 'not': 25,
 'are': 26,
 'been': 27,
 '”': 28,
 'but': 29,
 'who': 30,
 'this': 31,
 'had': 32,
 'they': 33,
 'after': 34,
 'i': 35,
 'were': 36,
 'we': 37,
 'will': 38,
 'about': 39,
 'one': 40,
 'or': 41,
 'isis': 42,
 'video': 43,
 'which': 44,
 'she': 45,
 'apple': 46,
 'state': 47,
 'up': 48,
 'her': 49,
 'would': 50,
 'their': 51,
 'more': 52,
 'also': 53,
 'when': 54,
 'told': 55,
 'new': 56,
 'out': 57,
 'no': 58,
 'all': 59,
 'people': 60,
 'there': 61,
 'you': 62,
 'its': 63,
 'if': 64,
 'him': 65,
 'islamic': 66,
 'news': 67,
 'man': 68,
 'could': 69,
 'what': 70,
 'year': 71,
 'watch': 72,
 'time': 73,
 'al': 74,
 'over': 75,
 'some': 76,
 'group': 77,
 'according': 78,
 'u': 79,
 's': 80,
 'into': 81,
 'firs

In [20]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = []
articles = []

In [0]:
from nltk.tokenize import sent_tokenize


In [0]:
texts=dataset['articleBody'].to_list()

In [0]:
articles = []

In [31]:
len(texts)

49972

In [0]:
for i in range(0,len(texts)):
  articles.append(sent_tokenize(texts[i]))


In [33]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

## Check 2:

first element of texts and articles should be as given below. 

In [0]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [0]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
import numpy as np

In [0]:
data=np.zeros((49972, MAX_SENTS,MAX_SENT_LENGTH),dtype=int)

In [36]:
data.shape

(49972, 20, 20)

In [0]:
dict_words=t.word_index.items()

In [47]:
t.word_index.items()

dict_items([('the', 1), ('to', 2), ('a', 3), ('of', 4), ('in', 5), ('and', 6), ('that', 7), ('is', 8), ('was', 9), ('on', 10), ('for', 11), ('said', 12), ('he', 13), ('with', 14), ('it', 15), ('his', 16), ('have', 17), ('as', 18), ('by', 19), ('has', 20), ('at', 21), ('from', 22), ('be', 23), ('an', 24), ('not', 25), ('are', 26), ('been', 27), ('”', 28), ('but', 29), ('who', 30), ('this', 31), ('had', 32), ('they', 33), ('after', 34), ('i', 35), ('were', 36), ('we', 37), ('will', 38), ('about', 39), ('one', 40), ('or', 41), ('isis', 42), ('video', 43), ('which', 44), ('she', 45), ('apple', 46), ('state', 47), ('up', 48), ('her', 49), ('would', 50), ('their', 51), ('more', 52), ('also', 53), ('when', 54), ('told', 55), ('new', 56), ('out', 57), ('no', 58), ('all', 59), ('people', 60), ('there', 61), ('you', 62), ('its', 63), ('if', 64), ('him', 65), ('islamic', 66), ('news', 67), ('man', 68), ('could', 69), ('what', 70), ('year', 71), ('watch', 72), ('time', 73), ('al', 74), ('over', 75

In [80]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
  if word=='Nicaragua'.lower():
    print(i)
    break

1557


The Below COde will work but it will take some time to execute on colab , if we have GPU supported it will owrk with out an issue.

In [0]:
for k,article in enumerate(articles):
  print('article',k)
  for i,sentence in enumerate(article):
    if i==20:
      break
    if i<20:
      for j,word in list(enumerate(sentence.split()))[:20]:
        for word_idx, i_idx in t.word_index.items():
          if word.lower()==word_idx:
            data[k,i,j]=i_idx
            break
    

In [0]:
#writing the loop is taking a lot of time to convert into index. but the above code block will work and I managed to get 4000 articles information
#I manually interrupted the kernel to save the time .
#I am going to use text_to_sequence for the faster execution

In [0]:
data=t.texts_to_sequences(texts)

In [120]:
data[0]

array([    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
        3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
         758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
        1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
         555,   235,    85,    89,  1067,  4115,  2349,    12,     3,
        1092,  3306,    19,     1,    89,     2,  1793,     1,   521,
        2009,    15,     9,     3,  3111,   481,   427,     7,   334,
           2,    17,   266,   129,    24,   972,     7,     9,  1980,
         516,     2,   778,   181,  3640,   972,   200,  2556,    44,
        6775,  1722,  1252,     5, 13317, 17936,     1,   778,    31,
         740,  3990,    67,    85,  2349,    12,  1557,    38,  1094,
         351,   775,     2,   367,   260,  1770,     5,  4450,    70,
         494,     1,   700,   189,    19,     1,   427,    32,     3,
        7417,     4,  2159,  1252,     6,     3,  5270,     4,  1217,
        1252,    12,

In [0]:
from tensorflow.python.keras.preprocessing import sequence

In [0]:
data = sequence.pad_sequences(data,maxlen=MAX_SENTS*MAX_SENT_LENGTH,padding='post')

In [0]:
data=np.reshape(data,(len(data), MAX_SENTS,MAX_SENT_LENGTH))

In [165]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
          758,    95],
       [ 1047,     3,  2679,  1752,     7,   189,     3,  1217,  1075,
         2030,   700,   159,     1,  3032,   448,     1,   555,   235,
           85,    89],
       [ 1067,  4115,  2349,    12,     3,  1092,  3306,    19,     1,
           89,     2,  1793,     1,   521,  2009,    15,     9,     3,
         3111,   481],
       [  427,     7,   334,     2,    17,   266,   129,    24,   972,
            7,     9,  1980,   516,     2,   778,   181,  3640,   972,
          200,  2556],
       [   44,  6775,  1722,  1252,     5, 13317, 17936,     1,   778,
           31,   740,  3990,    67,    85,  2349,    12,  1557,    38,
         1094,   351],
       [  775,     2,   367,   260,  1770,     5,  4450,    70,   494,
            1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159],
       [ 1

### Check 3:

Accessing first element in data should give something like given below.

In [0]:
data[0, :, :]

array([[    3,   487,   474,  7113,    79,     3,  3687,   325,     5,
         4200,   361,     4,  1525,  2913,     1,    89,    12,   451,
            0,     0],
       [  743,    96,  1044,     3,  2814,  1759,     7,   186,     3,
         1219,  1070,  1987,   736,   154,     1,  2990,   458,     1,
          543,   232],
       [   89,  1052,  4057,  2314,    12,     3,  1073,  3248,    19,
            1,    89,     2,  1751,     1,   518,  1980,    15,     9,
            3,  2879],
       [  182,  3691,   976,   196,  2515,    42,  6688,  1691,  1227,
            5, 13011, 17379,     1,   762,    30,   722,  3931,    66,
           87,     0],
       [ 2314,    12,  1882,    38,  1076,   346,   793,     2,   356,
          261,  1782,     5,  4396,    67,   486,     0,     0,     0,
            0,     0],
       [    1,   736,   186,    19,     1,   474,    32,     3,  7307,
            4,  2122,  1227,     6,     3,  5195,     4,  1219,  1227,
           12,  3308],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = []
articles_heading = []

In [0]:
data_head=np.reshape(data,(len(data), MAX_SENTS,MAX_SENT_LENGTH))

In [0]:
texts_head=dataset['Headline'].to_list()

In [0]:
for i in range(0,len(texts_head)):
  articles_heading.append(sent_tokenize(texts[i]))


The Below COde will work but it will take some time to execute on colab , if we have GPU supported it will owrk with out an issue.

In [0]:
for k,article in enumerate(articles_heading):
  print('article',k)
  for i,sentence in enumerate(article):
    if i==1:
      break
    if i<20:
      for j,word in list(enumerate(sentence.split()))[:20]:
        for word_idx, i_idx in t.word_index.items():
          if word.lower()==word_idx:
            data_head[k,i,j]=i_idx
            break
    

In [0]:
#writing the loop is taking a lot of time to convert into index. but the above block will work and I managed to get 4000 articles information
#I manually interrupted the kernel to save the time .
#I am going to use text_to_sequence for the faster execution

In [0]:
texts_head=dataset['Headline'].to_list()
data_head=t.texts_to_sequences(texts_head)
data_head = sequence.pad_sequences(data_head,maxlen=MAX_SENTS_HEADING*MAX_SENT_LENGTH,padding='post')

In [0]:
data_head=np.reshape(data_head,(len(data_head), MAX_SENTS_HEADING,MAX_SENT_LENGTH))

In [168]:
print(data_head.shape)

(49972, 1, 20)


### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels=pd.get_dummies(dataset['Stance'])

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [170]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
labels=np.asarray(labels)

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [179]:
labels[0]

array([0, 0, 0, 1], dtype=uint8)

In [0]:
## shuffle the data
data = data[indices]
data_head = data_head[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

In [0]:
#Here instead of splitting X and y , I am splitting article data and headline data in the first step
#and in the second step , I am splitting article data and label
#since I am using the same random state , the splitting will not differ in the both steps and I will get the proper labels as well

In [0]:
X_train_article, X_test_article, X_train_head, X_test_head = train_test_split(
    data,
    data_head,
    test_size=0.2,
    random_state=42
)

In [187]:
X_train_article[0]

array([[ 4365, 12578,  2158,     6,  9430,  5651,  1855,   217,   211,
            1,   447,  1971,     4,   930,   715,  2918,    19,     1,
         1971,     4],
       [12579,  2799,   702,   621,  9173,    21,   214,    10,     1,
          300,   100,  6807,    31,    71,     3,   738,  1433,     7,
            1,  6700],
       [ 5600, 12043,    32, 12580,    24,  1655,   771,   702,   621,
         2293,  3381,  1821,   117,    19,  1253,   120,   866,   828,
            8,   793],
       [   58,  1233,    70,  1736,    15,     8,  6161,  6162,  1150,
          148,   306,  6163,    91,  8861, 12581, 12582, 12583,   428,
          600,   200],
       [    1,   234,   164,    25,   640,   515,    41,  1603,     1,
         2134,     4,     1,  1114,  1225,     1,   420,     5,     3,
        10935,     1],
       [ 2393, 12584,    31,   480,    54,  1253,  1127,   939,     3,
         4785,    29,   104,  2292,    15,   953,     2,   456,  3094,
         1068,   273],
       [ 1

In [0]:
X_train_article, X_test_article, y_train, y_test= train_test_split(
    data,
    labels,
    test_size=0.2,
    random_state=42
)

In [189]:
X_train_article[0]

array([[ 4365, 12578,  2158,     6,  9430,  5651,  1855,   217,   211,
            1,   447,  1971,     4,   930,   715,  2918,    19,     1,
         1971,     4],
       [12579,  2799,   702,   621,  9173,    21,   214,    10,     1,
          300,   100,  6807,    31,    71,     3,   738,  1433,     7,
            1,  6700],
       [ 5600, 12043,    32, 12580,    24,  1655,   771,   702,   621,
         2293,  3381,  1821,   117,    19,  1253,   120,   866,   828,
            8,   793],
       [   58,  1233,    70,  1736,    15,     8,  6161,  6162,  1150,
          148,   306,  6163,    91,  8861, 12581, 12582, 12583,   428,
          600,   200],
       [    1,   234,   164,    25,   640,   515,    41,  1603,     1,
         2134,     4,     1,  1114,  1225,     1,   420,     5,     3,
        10935,     1],
       [ 2393, 12584,    31,   480,    54,  1253,  1127,   939,     3,
         4785,    29,   104,  2292,    15,   953,     2,   456,  3094,
         1068,   273],
       [ 1

In [0]:
#Splitting of two different inputs and outputs are done properly by keeping random_state

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [265]:
print(X_train_article.shape)
print(y_train.shape)

print(X_test_article.shape)
print(y_test.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [91]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(t.word_index)+1, 100))


for word, i in t.word_index.items():
	
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
#here we cant go for sequential model approach , as we have two different inputs 
#hence we should go for functional APIs as there will be two input layers and two enbedding layers one for each input and two LSTM s one for each embed layed
#And Finally dense layer with 4 activation units.

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense,Concatenate,Flatten,Dropout

from keras.models import Model
from keras import layers


### Model

In [0]:
article_input = Input(shape=(None,), dtype='int32', name='article_input')
head_input = Input(shape=(None,), dtype='int32', name='head_input')


In [0]:
shared_embed = Embedding(len(t.word_index) + 1,100,weights=[embedding_matrix],trainable=False)


In [0]:
article_embed = shared_embed(article_input)
head_embed = shared_embed(head_input)


In [0]:
article_lstm = LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='article_lstm')(article_embed)
head_lstm = LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='head_lstm')(head_embed)


In [0]:
conc = layers.concatenate([article_lstm,head_lstm],axis=-1)


In [0]:
dense = Dense(100,activation='relu')(conc)
dense = Dropout(0.3)(dense)
dense = Dense(4,activation='softmax')(dense)


In [0]:
model = Model(inputs=[article_input,head_input], outputs=[dense])


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])


In [261]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
article_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
head_input (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 100)    2787400     article_input[0][0]              
                                                                 head_input[0][0]                 
__________________________________________________________________________________________________
article_lstm (LSTM)             (None, 64)           42240       embedding_10[0][0]         

### Compile and fit the model

In [266]:
    model.fit([X_train_article.reshape(39977, 400),X_train_head.reshape(39977, 20)],[y_train],validation_data = ([X_test_article.reshape(9995,400),X_test_head.reshape(9995,20)],y_test),epochs=2, batch_size=128,verbose = True)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 39977 samples, validate on 9995 samples
Epoch 1/2
39977/39977 [==============================] - 197s 5ms/step - loss: 0.8127 - acc: 0.7280 - val_loss: 0.7541 - val_acc: 0.7323
Epoch 2/2
39977/39977 [==============================] - 192s 5ms/step - loss: 0.7506 - acc: 0.7365 - val_loss: 0.7337 - val_acc: 0.7350


In [280]:
loss,acc=model.evaluate([X_test_article.reshape(9995,400),X_test_head.reshape(9995,20)],y_test)


9995/9995 [==============================] - 37s 4ms/step


In [281]:
acc

0.7349674837716882

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)